In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
import time
from openpyxl import load_workbook

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd, numpy as np

import nbimporter
from pipeline import fn_acc_record
import warnings
warnings.filterwarnings('ignore')

In [10]:
def idx_find(L, text,jump):
    for idx, value in enumerate(L):
        if text in str(value):
            get_idx = L.index(value)
            return get_idx + jump
    return -1

In [11]:
def raw_cleaner(L):
    # if not any data, assign nan value
    L.append(np.nan)    
    
    if any("components" in str(item) for item in L):
        IlkYariSonucu = '-'
        MacSonucu = '-' 
    else:
        IlkYariSonucu = f"{L[5].split(' ')[1]}-{L[5].split(' ')[3]}"[:-1] 
        MacSonucu = f"{L[2]}-{L[4]}"
       
    
    cleaned = {
        'MacTarihi':L[idx_find(L, 'Hafta',0)].split('|')[0],
        'Lig':L[idx_find(L,'Hafta',0)].split('|')[1],
        'EvSahibi':L[0],
        'KonukEkip':L[1],
        'IlkYariSonucu': IlkYariSonucu,
        'MacSonucu': MacSonucu,
        
        'Ms1': L[idx_find(L, 'MAÇ SONUCU',2)],
        'Ms0':L[idx_find(L, 'MAÇ SONUCU',4)],
        'Ms2':L[idx_find(L, 'MAÇ SONUCU',6)],
        
        'CS10':L[idx_find(L, 'ÇIFTE ŞANS',2)],
        'CS12':L[idx_find(L, 'ÇIFTE ŞANS',4)],
        'CS02':L[idx_find(L, 'ÇIFTE ŞANS',6)],
        
        'H10Ms1':L[idx_find(L, 'HND. MS (1:0)',2)],
        'H10Ms0':L[idx_find(L, 'HND. MS (1:0)',4)],
        'H10Ms2':L[idx_find(L, 'HND. MS (1:0)',6)],
        'H20Ms1':L[idx_find(L, 'HND. MS (2:0)',2)],
        'H20Ms0':L[idx_find(L, 'HND. MS (2:0)',4)],
        'H20Ms2':L[idx_find(L, 'HND. MS (2:0)',6)],
        'H30Ms1':L[idx_find(L, 'HND. MS (3:0)',2)],
        'H30Ms0':L[idx_find(L, 'HND. MS (3:0)',4)],
        'H30Ms2':L[idx_find(L, 'HND. MS (3:0)',6)],
        'H01Ms1':L[idx_find(L, 'HND. MS (0:1)',2)],
        'H01Ms0':L[idx_find(L, 'HND. MS (0:1)',4)],
        'H01Ms2':L[idx_find(L, 'HND. MS (0:1)',6)],
        'H02Ms1':L[idx_find(L, 'HND. MS (0:2)',2)],
        'H02Ms0':L[idx_find(L, 'HND. MS (0:2)',4)],
        'H02Ms2':L[idx_find(L, 'HND. MS (0:2)',6)],
        'H03Ms1':L[idx_find(L, 'HND. MS (0:3)',2)],
        'H03Ms0':L[idx_find(L, 'HND. MS (0:3)',4)],
        'H03Ms2':L[idx_find(L, 'HND. MS (0:3)',6)],
        
        'Ms1den1':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',2)],
        'Ms0den1':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',4)],
        'Ms2den1':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',6)],       
        'Ms1den0':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',8)],
        'Ms0dan0':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',10)],
        'Ms2den0':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',12)],
        'Ms1den2':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',14)],
        'Ms0dan2':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',16)],
        'Ms2den2':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',18)],
        
        'Alt15':L[idx_find(L, '1,5 ALT/ÜST',2)],
        'Ust15':L[idx_find(L, '1,5 ALT/ÜST',4)],
        'Alt25':L[idx_find(L, '2,5 ALT/ÜST',2)],
        'Ust25':L[idx_find(L, '2,5 ALT/ÜST',4)],
        'Alt35':L[idx_find(L, '3,5 ALT/ÜST',2)],
        'Ust35':L[idx_find(L, '3,5 ALT/ÜST',4)],
        'Alt45':L[idx_find(L, '4,5 ALT/ÜST',2)],
        'Ust45':L[idx_find(L, '4,5 ALT/ÜST',4)],
        
    }
    # delete of fake last Nan value (-1)
    L.pop()
    
    df = pd.DataFrame([cleaned])
    return df

In [13]:
!pip install webdriver_manager


In [12]:
# Geckodriver yolu
geckodriver_path = "../set/geckodriver"

# Firefox service nesnesini oluştur
service = FirefoxService(executable_path=geckodriver_path)


# Firefox tarayıcısını service ile başlat
driver = webdriver.Firefox(service=service)
# Hedef web sayfasına git

macKolikUrl="https://www.mackolik.com/futbol/canli-sonuclar"
driver.get(macKolikUrl)

#time.sleep(5)
wait = WebDriverWait(driver, 10)
cookies=wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]')))
cookies.click()

# clock=wait.until(EC.element_to_be_clickable((By.XPATH, "//span[.='Saat']")))
# clock.click()

dateIcon = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.widget-dateslider__datepicker-toggle')))

# Click on the element
dateIcon.click()
butunTakvim = wait.until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]')))
html_contentTakvim = butunTakvim.get_attribute('outerHTML')
# print(html_contentTakvim)



# BeautifulSoup ile HTML içeriğini parse et
soup = BeautifulSoup(html_contentTakvim, 'html.parser')

# Günü ve Yılı al
day = int(soup.find('td', {'class': 'widget-datepicker__calendar-body-cell--selected-day'}).text.strip())
year = int(soup.find('div', {'class': 'widget-datepicker__selector--year'}).find('div', {'class': 'widget-datepicker__value'}).text.strip())

#print(f"day: {day}, year: {year}")

current_year = int(driver.find_element(By.XPATH, '/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[2]/div[2]').text)
previous_year_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='widget-dateslider__datepicker widget-dateslider__datepicker--open']/div/div/div[2]/div[1]")))

# Hedef yılı belirle (örneğin, 2020)
target_year = 2022

# Hedef yıla kadar önceki yıl düğmesine tıkla
while current_year > target_year:
    #print(str(target_year)+ "target year")
    previous_year_button.click()
    #time.sleep(1)
    current_year -= 1
    #print(current_year)
    # Yeni yılı kontrol et
    current_year = int(driver.find_element(By.XPATH, '/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[2]/div[2]').text)
    #print(current_year)

    
# calc of months
previous_month_button= wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[1]/div[1]")))
#wait = WebDriverWait(driver, 10)  # Adjust the timeout as needed
element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[1]/div[2]")))
month_str = element.text.strip()[:3]

#print(month_str)
# Map the month abbreviation to a numeric value
month_mapping = {'OCA': 1, 'ŞUB': 2, 'MAR': 3, 'NIS': 4, 'MAY': 5, 'HAZ': 6, 'TEM': 7, 'AĞU': 8, 'EYL': 9, 'EKI': 10, 'KAS': 11, 'ARA': 12}
#print(month_str+"month_str")
month_numeric = month_mapping.get(month_str)

target_month=12
month_numeric = month_mapping.get(month_str)


# Loop until you reach the target month
while month_numeric != target_month:
    #print(str(month_numeric)+"month_numeric")
    #print(str(target_month)+"target_month")

    previous_month_button.click()
    #time.sleep(1)
    
    element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[1]/div[2]")))
    month_str = element.text.strip()[:3]
    #print(month_str+"month_str")
    month_numeric = month_mapping.get(month_str)
    #print(month_numeric)
    #print(target_month)
    
target_day=10
day_element = driver.find_element(By.XPATH, f"//td[@class='widget-datepicker__calendar-body-cell' and text()='{target_day}']")
#time.sleep(19)
day_element.click()
time.sleep(3)


container_xpath = "//div[@class='match-row__match-content']"  #(//div[@class='match-row__match-content'])[1]

# Container elementini bul
allLinksElements = driver.find_elements(By.XPATH, container_xpath)

# df_all = pd.DataFrame()
count = 0
# result = []

# dataframe_listesi = []
while True:
    try:
        # Container elementini bul
        allLinksElements = driver.find_elements(By.XPATH, container_xpath)
        

        # If all links are processed, break out of the loop
        if not allLinksElements:
            break
        
        # Iterate through each element and click
        for i, element in enumerate(allLinksElements, start=1):
            result = []
            try:
                dynamic_xpath = f"(//div[@class='match-row__match-content'])[{i}]"
                elementx = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, dynamic_xpath)))
                elementx = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, dynamic_xpath)))

                print(f"(try Worked... {str(i)})")
                element.click()

                # Switch to the newly opened tab (assuming it's the last tab)
                driver.switch_to.window(driver.window_handles[-1])

                print(f"(went new page... {str(i)})")
                time.sleep(1)
                # Perform actions in the new tab
                iddiaButton = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[1]/div[2]/div/a[3]")))

                tx = iddiaButton.get_attribute('class')
                # Check if the iddiaButton is disabled
                if "disabled" not in tx:
                    
                    print('Button is not disabled. Proceeding...')
                    iddiaButton.click()
                    time.sleep(1)
                    
                    ###############
                    macScorboard = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='p0c-soccer-match-details-header']")))
                    macDetailsData = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[2]/main/div[1]/div[2]/div/div[2]/div")))
                    
                    skorbord = macScorboard.text
                    
                    lines = skorbord.split('\n')
                    for line_sb in lines:
                        result += [line_sb]
                        # result.append(line)
                                                        
                    odds = macDetailsData.text                    
                    lines_odds = odds.split('\n')
                    
                    for line_gn in lines_odds:
                        result += [line_gn]
                        
                        # result.append(line)

                    
                    df = pd.DataFrame(result, columns=[0])
                    df.to_excel(f'./data/new_data_raw_{i}.xlsx')
                    df_cleaned_game = raw_cleaner(list(df[0]))
                    
                    # df_cleaned_game.to_excel('df_all_cleaned_games.xlsx')
                    fn_acc_record('./data/df_all_cleaned_games.xlsx', df_cleaned_game, header=False, index=False)
                    print(f'***** data cleaned_{i} ****** ')
                    
                    
                    # Close the current tab
                    driver.close()
                    print(f"(gone new page closed.. {str(i)})")
                    print('-----------------------------------------------')
                    time.sleep(3)
                    # Switch back to the original tab
                    driver.switch_to.window(driver.window_handles[0])

                    time.sleep(3)
                else:
                    print('Button is disabled. Skipping...')
                    # Switch back to the original tab
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            except StaleElementReferenceException:
                count += 1
                print('<><><><><><', count)
                print("Stale Element Reference Exception. Retrying.")
                continue

    except StaleElementReferenceException:
        print("Stale Element Reference Exception during initial find. Retrying after refreshing the page.")
        driver.refresh()
        continue
    print('#### Program has been run, Status: Successful ####')
    break

NoSuchDriverException: Message: Unable to locate or obtain driver for firefox; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
